In [3]:
import numpy as np
from matplotlib    import pyplot as plt
from numpy.random  import default_rng
from tqdm          import tqdm
from time          import perf_counter 
from scipy.stats   import pearsonr, poisson, wishart, multivariate_normal, norm, vonmises, multivariate_t, laplace
from scipy.special import gamma, factorial, gammaln
from scipy.special import i0 as I0
from dw_tools      import bivariate_tools

## Conditional joint probability distribution of $(E_1, E_2)$ given $E_3$

### Acentric case
Following https://en.wikipedia.org/wiki/Multivariate_normal_distribution#Conditional_distributions, 

$$
P\left(E_1,E_2 | E_3\right) = P\left(E_{1x},E_{2x},E_{1y},E_{2x}|E_{3x}, E_{3y}\right)=N\left(r E_3,C_{1,2|3}\right)
$$

(note the rearrangement of rows), with conditional covariance matrix

$$
C_{1,2|3} = \frac{1}{2}
\begin{bmatrix}
1   -r^2 & r_x -r^2 & 0       & 0        \\
r_x -r^2 & 1   -r^2 & 0       & 0        \\
0        & 0        & 1  -r^2 & r_x -r^2 \\
0        & 0        & r_x-r^2 & 1 -r^2  
\end{bmatrix}
$$

**Notes**
- $r$ and $r_x$ cannot adopt arbitrary combinations of values. Specifically, the conditional covariance matrix is only positive definite if $1+r_x \geq 2r^2$. 
- $E_1$ and $E_2$ are conditionally independent (and uncorrelated) when $r_x = r^2$. In that case, $1+r_x = 1+r^2 \geq 2r^2 $.
- In our case, $r_x$ or ```rx``` is the correlation between components of the ON and OFF structure factors. $r$ or ```r``` is the correlation between ON or OFF structure factors and the reference data. The prior can be made irrelevant by setting $r=0$.
- Setting both $r$ and $r_x$ to 0 reduces the prior to the product of Wilson distributions.
- Previous analysis suggests $r \sim 0.9$ and $r_x > 0.98$ as typical values. 
- As we saw previously, $r$ and $r_x$ are generally functions of resolution that can be well-described by forms $r=a e^{-bs^2}$. For an ON/OFF data set, one could estimate $a, b, a_x, b_x$ by maximum likelihood.

### Centric case
Following https://en.wikipedia.org/wiki/Multivariate_normal_distribution#Conditional_distributions, 

$$
P\left(E_{1c},E_{2c} | E_{3c}\right) = N\left(r E_{3c},C_{1c|3c}\right)
$$

with 

$$
C_{1c|3c} = 
\begin{bmatrix}
1   -r^2 & r_x -r^2 \\
r_x -r^2 & 1   -r^2         
\end{bmatrix}
$$

We then have that 

$$
P\left(|E_{1c}|,|E_{2c}| | |E_{3c}|\right) = P\left(E_{1c},E_{2c} | E_{3c}\right) + 
                                             P\left(-E_{1c},E_{2c} | E_{3c}\right) +
                                             P\left(E_{1c},-E_{2c} | E_{3c}\right) +
                                             P\left(-E_{1c},-E_{2c} | E_{3c}\right)
$$

There should not be any need to further simplify this.

## Implementation
The bivariate Rice distribution and 2D folded normal are validated in **6_Bivariate_priors_tidy**.

In [4]:
E1 = 1
E2 = 2
E3 = 2
r  = 0.0
rx = 0.99

In [5]:
print(bivariate_tools.FoldedNorm2D_wrapper(  E1=E1, E2=E2, rx=rx, E3=E3, r=r, Sigma=1.0)) # acentric
print(bivariate_tools.Bivariate_Rice_wrapper(E1=E1, E2=E2, rx=rx, E3=E3, r=r, Sigma=1.0)) #  centric

1.3852087261928682e-11
2.289987096433528e-22


### Performance

In [6]:
nx, ny = (200,200)
xy_max = 6
xbase  = np.linspace(0.001, xy_max, nx)
ybase  = np.linspace(0.001, xy_max, ny)
xx,yy  = np.meshgrid(xbase,ybase)
y_in   = np.transpose(np.array([xx.flatten(), yy.flatten()]))
print(y_in.shape)

t1_start = perf_counter()  
result_ac = np.zeros((nx,ny))
result_c  = np.zeros((nx,ny))
for i in range(nx):
    for j in range(ny):
#         result_ac[i,j] = bivariate_tools.Bivariate_Rice_wrapper(xbase[i],ybase[j],rx,E3,r)
        result_c[ i,j] = bivariate_tools.FoldedNorm2D_wrapper(    xbase[i],ybase[j],rx,E3,r)
t1_end = perf_counter()  

print(f"Elapsed time: {t1_end-t1_start:.3} s")
print(f"Integrated probability density (acentric): {np.sum(result_ac[:])*((xy_max/nx)*(xy_max/ny)):.4}")
print(f"Integrated probability density (acentric): {np.sum(result_c[ :])*((xy_max/nx)*(xy_max/ny)):.4}")

(40000, 2)
Elapsed time: 12.2 s
Integrated probability density (acentric): 0.0
Integrated probability density (acentric): 0.9934


On my laptop, it took 1.7 sec for 40,000 acentric calculations, and 13.3 sec for centric calculations. Neither functions is vectorized. There is, however, a vectorized version of the 2D folded normal in ```bivariate_tools.py```:

In [7]:
t1_start = perf_counter()  
result_c = bivariate_tools.FoldedNorm2D_vect_wrapper(y_in[:,0].reshape(-1,1), y_in[:,1].reshape(-1,1),rx,E3,r)
result_c = result_c.reshape(nx,ny)
t1_end = perf_counter()  

print(f"Elapsed time: {t1_end-t1_start:.3} s")
print(f"Integrated probability density (acentric): {np.sum(result_c[ :])*((xy_max/nx)*(xy_max/ny)):.4}")

Elapsed time: 0.0156 s
Integrated probability density (acentric): 0.9934
